Ordering by complexity to keep feasible:

- [ ] Minimum viable - assign subset of residents to subset of rotations with no requirements or maximization as reliable test bed
- [ ] Implement preference maximization with repeatable known output
- [ ] Implement residents required last - requires "full beans" inputs to give enough blocks to fit requirements

If promising results:
- [ ] Implement Madigan/FM handling

In [ ]:
import pandas as pd
from ortools.sat.python import cp_model
from icecream import ic

In [ ]:
names = pd.read_excel("inputs.xlsx", sheet_name="resident names").head() #sampled

In [ ]:
rotations = pd.read_excel("inputs.xlsx", sheet_name="rotations")

In [ ]:
blocks = pd.read_excel("inputs.xlsx", sheet_name="blocks")

In [ ]:
model = cp_model.CpModel()
model.SetName("Residency Scheduler")

x = {}
for n in names.name:
    for r in rotations.name:
        for b in blocks.block:
            x[n,r,b] = model.NewBoolVar(f"{n} - {r} - Block {b} | interval:")



print(model.ModelStats())

In [ ]:
rotations

In [ ]:
variables = []

model = cp_model.CpModel()
model.SetName("Residency Scheduler")

for n in names.name:
    for r in rotations.name:
        for b in blocks.block:
            var = model.NewBoolVar(f"{n} - {r} - Block {b} | interval:")
            variables.append((n, r, b, var))

model_var_tracker = pd.DataFrame(variables, columns=["Resident", "Rotation", "Block", "Variable Label"])

# Residents are always scheduled to something
for n in names.name:
    for b in blocks.block:
        # model.Add(
        #     sum(model_var_tracker.query("Resident == @n and Block == @b")["Variable Label"]) == 1
        # )
        model.AddExactlyOne(model_var_tracker.query("Resident == @n and Block == @b")["Variable Label"])

# Residents do rotations no more than a certain number of times
for n in names.name:
    for r in rotations.name:
        for b in blocks.block:
            rotations.query("name == @r")["maximum_repeats"].values[0]

# Don't schedule more residents on a rotation during each block than max
for r in rotations.name:
    for b in blocks.block:
        max_residents = rotations.query("name == @r")["maximum_residents"].values[0]  # Get max resident limit for rotation
        model.Add(
            sum(model_var_tracker.query("Rotation == @r and Block == @b")["Variable Label"]) <= max_residents
        )


print(model.ModelStats())

In [ ]:
rotations.query("name == @r")["maximum_repeats"].values[0]

In [ ]:
model_var_tracker.query("Rotation == @r and Block == @b")["Variable Label"]

In [ ]:
for r in rotations.name:
    print(rotations.query("name == @r")["maximum_residents"].values[0])
    break

In [ ]:
# Initialize the solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Process and display results
if status in (cp_model.FEASIBLE, cp_model.OPTIMAL):

    if status == cp_model.FEASIBLE:
        print(">> Feasible\n")
    elif status == cp_model.OPTIMAL:
        print(">> Optimal\n")
    assigned_rotations = []

    for _, row in model_var_tracker.iterrows():
        # if solver.Value(row["Variable Label"]) == 1:
        assigned_rotations.append((row["Resident"], row["Rotation"], row["Block"], solver.Value(row["Variable Label"])))
    # Convert results to a DataFrame for readability
    results_df = pd.DataFrame(assigned_rotations, columns=["Resident", "Rotation", "Block", "Scheduled"])

else:
    print("No feasible solution found.")

In [ ]:
results_df

In [ ]:
results_df.head(30)

In [ ]:
model_var_tracker

In [ ]:
print(model.ModelStats())

In [ ]:
model_var_tracker.query('Resident == @n & Rotation == @r & Block == @b')

In [ ]:
model_var_tracker.query("Resident == @n & Block == @b")["Variable Label"]